In [9]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

import os
os.chdir('/home/cat/code/manifolds/')

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange

from scipy.io import loadmat

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.spatial import ConvexHull, convex_hull_plot_2d

# 
np.set_printoptions(suppress=True)

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def connect2(ends):
    d0, d1 = np.abs(np.diff(ends, axis=0))[0]
    if d0 > d1: 
        return np.c_[np.linspace(ends[0, 0], ends[1, 0], d0+1, dtype=np.int32),
                     np.round(np.linspace(ends[0, 1], ends[1, 1], d0+1))
                     .astype(np.int32)]
    else:
        return np.c_[np.round(np.linspace(ends[0, 0], ends[1, 0], d1+1))
                     .astype(np.int32),
                     np.linspace(ends[0, 1], ends[1, 1], d1+1, dtype=np.int32)]

    
    # ####################################################
# ############## LOAD DATA ##############
# ####################################################
# root_dir = '/media/cat/4TB/donato/'
# animal_id = 'DON-006084'

# session = '20210519'

# # c = calcium.Calcium()
# # c.root_dir = root_dir
# # c.animal_id = animal_id
# # c.session = session
# # c.load_suite2p()


In [174]:
###############################################
############ LOAD MATCHED DATA ################
###############################################
from __future__ import print_function
import cv2
import numpy as np


                
# 
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'

clrs = ['black','red']
sessions = ['20210519', '20210520']
#sessions = ['20210520', '20210521']
#sessions = ['20210523', '20210524']

sess1 = sessions[0][-2:]
sess2 = sessions[1][-2:]

#
data = scipy.io.loadmat('/media/cat/4TB/donato/DON-006084/'+str(sess1)+'_'+str(sess2)+'.mat')
cell_ids = data['roiMatchData']['allSessionMapping'][0][0]

# convert from matlab to python data
cell_ids = cell_ids -1 
print ("# of matched cells: ", cell_ids.shape)

#fig=plt.figure()
imgs=[]
pts_array = []
all_pts = []
all_centres = []
all_footprints = []
for ctr, session in enumerate(sessions):
    
    # 
    stat = np.load(os.path.join(
                        root_dir,
                        animal_id,
                        session,
                        'suite2p/plane0/stat.npy'), allow_pickle=True)
    # 
    Fall = scipy.io.loadmat(os.path.join(
                            root_dir,
                            animal_id,
                            session,
                            'suite2p/plane0/Fall.mat'))

    # 
    iscell = np.int32(Fall['iscell'])
    print ("iscell: ", iscell.shape)

    # 
    idxs = np.where(iscell[:,0]==1)[0]
    
    print ("valid cells: ", idxs.shape, idxs[-1])
    
    img = np.zeros((512,512))
    pts3 = []
    pts4 = []
    pts5 = []
    pts6 = []
    
    # 
    
    
    # 
    all_points  = []
    for cell in cell_ids[:,ctr]:
        cell_valid = idxs[cell]
        
        # 
        points = np.vstack((stat[cell_valid]['xpix'],
                            stat[cell_valid]['ypix'])).T
        pts6.append(points)
        
        pts5.append(np.median(points,0))
        #.append(np.median(points,0))
        
        # 
        hull = ConvexHull(points)
        hull_ids = hull.vertices
        hull_points = points[hull.vertices]
        hull_points = np.vstack((hull_points, hull_points[0]))
        pts4.append(hull_points)

        

        #
        pts_local = []
        for p in range(hull_points.shape[0]-1):
            pts = np.vstack((hull_points[p],
                             hull_points[p+1]))
            pts = connect2(pts)
            img[pts[:,0],
                pts[:,1]]= 1

            pts_local.append(pts)
        pts3.append(np.vstack(pts_local))
        
        #
    
    all_footprints.append(np.array(pts6))
    pts_array.append(np.vstack(pts5))
    all_centres.append(np.vstack(pts5))
    all_pts.append(np.array(pts3))
    # 
    imgs.append(img)

print ("sess 1 sess 2 ", sess1, sess2)

# of matched cells:  (153, 2)
iscell:  (616, 2)
valid cells:  (441,) 607


/tmp/ipykernel_522703/3157560528.py:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_footprints.append(np.array(pts6))
/tmp/ipykernel_522703/3157560528.py:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_pts.append(np.array(pts3))


iscell:  (717, 2)
valid cells:  (521,) 713
sess 1 sess 2  19 20


In [170]:
print (len(all_pts[0]))
print (all_pts[0][0])

153
[[216 255]
 [216 254]
 [216 253]
 [216 253]
 [217 252]
 [218 251]
 [219 250]
 [220 249]
 [220 249]
 [221 248]
 [222 248]
 [222 248]
 [223 248]
 [224 248]
 [225 248]
 [225 248]
 [226 249]
 [226 250]
 [226 251]
 [227 252]
 [227 252]
 [227 253]
 [227 254]
 [227 255]
 [227 256]
 [227 256]
 [226 257]
 [225 258]
 [224 259]
 [224 259]
 [223 259]
 [222 259]
 [221 259]
 [221 259]
 [220 258]
 [219 258]
 [219 258]
 [218 257]
 [217 256]
 [216 255]]


In [38]:
##########################################################
##########################################################
##########################################################
sessions = ['20210519', '20210520']
#sessions = ['20210520', '20210521']
#sessions = ['20210523', '20210524']

sess1 = sessions[0][-2:]
sess2 = sessions[1][-2:]


centres_array = []
imgs=[]
pts_array = []

for ctr, session in enumerate(sessions):
    root_dir = '/media/cat/4TB/donato/'
    animal_id = 'DON-006084'

    # 
    c = calcium.Calcium()
    c.root_dir = root_dir
    c.animal_id = animal_id
    c.session = session
    c.load_suite2p()
    
    print (c.F.shape)
    img = np.zeros((512,512))
    
    # 
    pts5 = []
    all_points = []
    subsample = 1
    cell_ids = np.arange(0,c.F.shape[0],subsample)
    for cell in cell_ids:
    
        # 
        points = c.get_footprint_contour(cell)
        all_points.append(points)
        
        pts5.append(np.median(points,0))
        
        #
        for p in range(points.shape[0]-1):
            pts = np.vstack((points[p],
                             points[p+1]))
            pts = connect2(pts)
            img[pts[:,0],
                pts[:,1]]= 1

    centres_array.append(np.vstack(pts5))
    imgs.append(img)
    pts_array.append(all_points)

    



(531, 55740)


(584, 55740)


In [157]:
##########################################################
##########################################################
##########################################################
from tqdm import tqdm, trange

# compute x projection first
fig=plt.figure()
mean = 4
final_shifts=[]
final_xy = []
final_streches = []
for k in range(2):
    x1 = np.int32(pts_array[0][:,k])
    x2 = np.int32(pts_array[1][:,k])

    xc1 = np.zeros(512)
    xc1[x1]= 1
    xc1 = scipy.ndimage.gaussian_filter1d(xc1, mean,axis=0)

    xc2 = np.zeros(512)
    xc2[x2]=1
    xc2 = scipy.ndimage.gaussian_filter1d(xc2, mean,axis=0)

    # 
#     ax=plt.subplot(2,2,k+1)
#     plt.plot(xc1)
#     plt.plot(xc2)
    
    ###########################################
    # find best translations:
    corr = []
    shifts = np.arange(-100,100,1)
    for p in shifts:
        temp = np.roll(xc1,p)
        #cc = scipy.signal.correlate(temp, xc2, axis=0)
        cc = np.corrcoef(temp, xc2)
        
        corr.append(cc[0,1])
        
    #
    idx = shifts[np.argmax(corr)]
    final_shifts.append(idx)
    
    xc1_rolled = np.roll(xc1,idx)
    ax=plt.subplot(3,2,k+1)

    plt.plot(xc1, c='red')
    plt.plot(xc1_rolled,'--',c='red')
    plt.plot(xc2, c='blue')

    
    #
    ###########################################
    # find best stretch/shrink
    
    
    
    upsample = 10

    # 
    streches = np.arange(0.9,1.1,0.001)
    shifts2 = np.arange(-10,10,1)
    idx_s = []
    vals_s = []
    for s in tqdm(streches):
        xc_up = scipy.signal.resample(xc1_rolled, xc1_rolled.shape[0]*upsample)
        # pad it with a bunch of zeros
        xc_up = np.concatenate((xc_up, np.zeros(1000)))

        xlocs = np.arange(xc_up.shape[0])

        # 
        xlocs = np.int32(xlocs*s)
        idxx = np.where(xlocs>=xc_up.shape[0])[0]

        # if strech is longer than the original data, just clip the last bits
        if len(idxx)>0:
            xlocs=xlocs[:idxx[0]]

        #    
        xc_up_streched = xc_up[xlocs]
        xc_streched = xc_up_streched[::upsample][:512]
        
        #print ("xc_streched ", xc_streched.shape)

        # 
        corr = []
        for p in shifts2:
            
            #
            temp = np.roll(xc_streched,p)
            #cc = scipy.signal.correlate(temp, xc2, axis=0)
            cc = np.corrcoef(temp, xc2)

            corr.append(cc[0,1])
        
        idx = shifts2[np.argmax(corr)]
        
        idx_s.append(shifts2[idx])
        
        vals_s.append(corr[idx])

    #print ("idx_s: ", idx_s)
    #print ("vals_s: ", vals_s)

    agmax = np.argmax(vals_s)
    best_stretch = streches[agmax]
    best_final_shift = idx_s[agmax]
    print ("Best: ", best_stretch, best_final_shift)
    
    final_xy.append(best_final_shift)
    final_streches.append(best_stretch)
    
############################################
print ("Initial shifts: ", final_shifts, " final stretch: ", final_streches, ", last shifts: ", final_xy)
#final_shifts = np.array(final_shifts)//2
   
    
ax=plt.subplot(3,2,3)
plt.scatter(pts_array[0][:,1],pts_array[0][:,0], c = 'blue')
plt.scatter(pts_array[1][:,1],pts_array[1][:,0], c = 'red')

ax=plt.subplot(3,2,4)

pts11 = pts_array[0].copy()
pts22 = pts_array[1].copy()

pts11[:,0] = pts11[:,0]+final_shifts[0]
pts11[:,1] = pts11[:,1]+final_shifts[1]

plt.scatter(pts11[:,1],pts11[:,0], c = 'blue')
plt.scatter(pts22[:,1],pts22[:,0], c = 'red')


#
ax=plt.subplot(3,2,5)
plt.imshow(imgs[0]+imgs[1]*-1)

#
ax=plt.subplot(3,2,6)

############################################
# do initial translation
imgs1_s = imgs[0].copy()
imgs1_s = np.roll(imgs1_s, final_shifts[1],1)
if final_shifts[0]>0:
    imgs1_s[:,:final_shifts[0]]=0
else:
    imgs1_s[:, final_shifts[0]:]=0
# 
imgs1_s = np.roll(imgs1_s, final_shifts[0],0)
if final_shifts[1]>0:
    imgs1_s[:final_shifts[1]]=0
else:
    imgs1_s[final_shifts[1]:]=0

############################################
if False:
    # do stretch
    from skimage import data, color
    from skimage.transform import rescale, resize, downscale_local_mean

    imgs1_s = rescale(imgs1_s, np.array(final_streches)[::-1], anti_aliasing=False)
    temp = np.zeros((512,512))
    temp[:min(imgs1_s.shape[0],512),:min(imgs1_s.shape[1],512)] = imgs1_s[:min(imgs1_s.shape[0],512),:min(imgs1_s.shape[1],512)]

    ############################################
    # do final translations
    imgs1_s = temp.copy()
    imgs1_s = np.roll(imgs1_s, final_xy[1],1)
    #if final_shifts[0]>0:
    #    imgs1_s[:,:final_xy[0]]=0
    #else:
    #    imgs1_s[:, final_xy[0]:]=0
    # 
    imgs1_s = np.roll(imgs1_s, final_xy[0],0)
    #if final_xy[1]>0:
    #    imgs1_s[:final_xy[1]]=0
    #else:
    #    imgs1_s[final_xy[1]:]=0
    #plt.imshow(temp)
    
# 
plt.imshow(imgs1_s+imgs[1]*-1)
plt.suptitle("Initial shifts: "+ str(final_shifts)+ " final stretch: "
             +str(final_streches) + ", last shifts: " +str(final_xy))
plt.show()


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 201/201 [00:00<00:00, 272.56it/s]


Best:  1.0290000000000001 -1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 201/201 [00:00<00:00, 264.69it/s]


Best:  0.9980000000000001 -10
Initial shifts:  [-27, 54]  final stretch:  [1.0290000000000001, 0.9980000000000001] , last shifts:  [-1, -10]


In [105]:
fig=plt.figure()


cmap = plt.get_cmap("viridis", 20)

streches = np.arange(0.90,1.10,0.01)
for ctr,s in enumerate(streches):
    xc_up = scipy.signal.resample(xc1_rolled, xc1_rolled.shape[0]*upsample)
    # pad it with a bunch of zeros
    xc_up = np.concatenate((xc_up, np.zeros(1000)))
    
    xlocs = np.arange(xc_up.shape[0])

    # 
    xlocs = np.int32(xlocs*s)
    idxx = np.where(xlocs>=xc_up.shape[0])[0]

    # if strech is longer than the original data, just clip the last bits
    if len(idxx)>0:
        xlocs=xlocs[:idxx[0]]

    #    
    xc_up_streched = xc_up[xlocs]
    xc_streched = xc_up_streched[::upsample][:512]
    print (xc_streched.shape)
    plt.plot(xc_streched,
            color=cmap(ctr))
    
plt.plot(xc1_rolled,
        c='red',
        linewidth=2)   
plt.show()

(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)
(512,)


In [80]:
#######################################################
#######################################################
#######################################################
# run brute force cell contour matcher
contour_pts = np.int32(c.get_footprint_contour(0))


pts2 = []
for p in range(contour_pts.shape[0]-1):
    pts = np.vstack((contour_pts[p],
                     contour_pts[p+1]))
    pts = connect2(pts)
    pts2.append(pts)

    
contour_pts2 = np.int32(np.vstack(pts2))

#
fig=plt.figure()
ax=plt.subplot(1,2,1)
# original points
plt.scatter(contour_pts[:,0],contour_pts[:,1],
           c='red',
           s=100,
           alpha=.5)

# interpolated points
plt.scatter(contour_pts2[:,0],contour_pts2[:,1],
           c='black',
           alpha=.5)

#
ax=plt.subplot(1,2,2)

def intersect1d_searchsorted(A,B,assume_unique=False):
    if assume_unique==0:
        B_ar = np.unique(B)
    else:
        B_ar = B
    idx = np.searchsorted(B_ar,A)
    idx[idx==len(B_ar)] = 0
    return A[B_ar[idx] == A]


            
for k in trange(100):
    corr = find_neuron_correlation(contour_pts2, contour_pts2)

plt.imshow(corr)
plt.colorbar()

plt.show()



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 103.96it/s]


In [314]:
############################################
############################################
############################################

def find_neuron_correlation_contour(cell1, cell2):
    
    # 
    width_shift = 5
    width = 20
    corr = np.zeros((width*2,width*2))
    n_nodes = max(cell1.shape[0],cell2.shape[0])
    #print ("nnodes: ", n_nodes)
    for x in range(-width_shift,width_shift,1):
        for y in range(-width_shift,width_shift,1):
            temp1 = cell1.copy()
            temp1[:,0]+=x
            temp1[:,1]+=y

            #idx = np.intersect1d(cell2,temp1)
            
            # find overlapping points
            idx = np.where(np.isin(cell2,temp1).min(1))[0]
            
            # 
            corr[x+width,y+width] = idx.shape[0]/n_nodes
    
    return corr     

def find_neuron_correlation_footprint(cell1, cell2):

    #cell1 = np.int32(cell1)
    #cell2 = np.int32(cell2)
#     # do single run:
#     if False:
#         # 
#         n_nodes = max(cell1.shape[0],cell2.shape[0])

#         # 
#         img1 = np.zeros((512,512))
#         for c1 in cell1:
#             #print (c1)
#             img1[c1[0],c1[1]]+=1
#         for c2 in cell2:
#             img1[c2[0],c2[1]]+=1
#         idx=np.where(img1==2)[0]
#         #print ("idx", idx)
#         #print (idx.shape, n_nodes)
#         #adsf
        
#         # 
#         corr = idx.shape[0]/n_nodes   
    
    # look at footprints version 1.
    if False:
        # 
        n_nodes = max(cell1.shape[0],cell2.shape[0])
        img1 = np.zeros((512,512))
        for c1 in cell1:
            #print (c1)
            img1[c1[0],c1[1]]+=1
        for c2 in cell2:
            img1[c2[0],c2[1]]+=1
        idx=np.where(img1==2)[0]

        corr = idx.shape[0]/n_nodes
        
    # look at footprints version 2.
    if False:
        # 
        n_nodes = max(cell1.shape[0],cell2.shape[0])

        # 
        idx = np.where(np.isin(cell2,cell2).min(1))[0]

        corr = idx.shape[0]/n_nodes
    
    # look at contours ; use shifting
    if False:
        
        width_shift = 5
        width = 20
        corr = np.zeros((width*2,width*2))
        n_nodes = max(cell1.shape[0],cell2.shape[0])
        #print ("nnodes: ", n_nodes)
        for x in range(-width_shift,width_shift,1):
            for y in range(-width_shift,width_shift,1):
                temp1 = cell1.copy()
                temp1[:,0]+=x
                temp1[:,1]+=y

                #idx = np.intersect1d(cell2,temp1)

                # find overlapping points
                idx = np.where(np.isin(cell2,temp1).min(1))[0]

                # 
                corr[x+width,y+width] = idx.shape[0]/n_nodes
        
        corr = np.max(corr)
        
    # look at contours single run
    if True:
        
        n_nodes = max(cell1.shape[0],cell2.shape[0])
        counter =0
        for k in range(cell1.shape[0]):
            dist = np.abs(cell2 - cell1[k]).sum(1)
            if np.min(dist)==0:
                #print ("dist sums xy: ", dist)
                counter+=1
        #print ('counter: ', counter)
        corr = counter/n_nodes
               
    return corr     

###################################################
print ("# of cells in session 1: ", len(all_pts[0]), " , sess #2: ", len(all_pts[1]))
print ("# of cells in session 1: ", len(all_footprints[0]), " , sess #2: ", len(all_footprints[1]))

#cells1 = all_pts[0]
#cells2 = all_pts[1]
cells1 = all_footprints[0]
cells2 = all_footprints[1]

# 
corrs = np.zeros((len(all_pts[0]), 
                  len(all_pts[1])))
print (corrs.shape) 
for c1 in trange(cells1.shape[0]):
    for c2 in range(c1+1, cells2.shape[0],1):
        
        if True:
            temp1 = all_pts[0][c1]
            temp1 = temp1 - np.median(temp1,axis=0)
            temp2 = all_pts[1][c2]
            temp2 = temp2 - np.median(temp2,axis=0)
            corr = find_neuron_correlation_footprint(temp1, temp2)
        
        if False:
            temp1 = all_footprints[0][c1]
            temp1 = temp1 - np.median(temp1,axis=0)
            temp2 = all_footprints[1][c2]
            temp2 = temp2 - np.median(temp2,axis=0)
            #print (temp1)
            
            corr = find_neuron_correlation_footprint(temp1, temp2)
        
        # print (c1,c2, corr)
        corrs[c1,c2]= corr

#plt.show()


# of cells in session 1:  153  , sess #2:  153
# of cells in session 1:  153  , sess #2:  153
(153, 153)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153/153 [00:11<00:00, 13.88it/s]


In [313]:
##############################################
##############################################
##############################################

cell1 = all_pts[0][1]
cell1 = cell1 - np.median(cell1,axis=0)
cell2 = all_pts[1][3]
cell2 = cell2 - np.median(cell2,axis=0)
cell1 = np.int32(cell1)
cell2 = np.int32(cell2)
corr = find_neuron_correlation_footprint(cell1, cell2)
print ("corr: ", corr)

n_nodes = max(cell1.shape[0],cell2.shape[0])
counter =0
for k in range(cell1.shape[0]):
    dist = np.abs(cell2 - cell1[k]).sum(1)
    if np.min(dist)==0:
        #print ("dist sums xy: ", dist)
        counter+=1
#print ('counter: ', counter)
corr = counter/n_nodes
#print ("n_nodes: ", n_nodes, cell1.shape, cell2.shape, "corr: ", corr)

fig = plt.figure()
plt.scatter(cell1[:,0],
            cell1[:,1],
           alpha=.5)
plt.scatter(cell2[:,0],
            cell2[:,1],
           alpha=.5)
plt.show()

corr:  1.0
dist sums xy:  [13 12 11 10  9  8  9 10 10 10  9  9  8  7  7  6  5  5  3  1  1  0  1  2
  3  4  5  6  7  8  8  9 11 12 12 13 13 14 14 14 14 14 13 13]
dist sums xy:  [10 11 12 13 14 15 16 17 17 17 16 16 15 14 14 13 12 12 10  8  8  7  6  5
  4  3  2  1  0  1  1  2  4  5  5  6  6  7  7  7  7  7  8 10]
dist sums xy:  [10 11 12 13 14 15 16 17 17 17 16 16 15 14 14 13 12 12 10 10 10  9  8  7
  6  5  4  3  2  1  1  0  2  3  3  4  4  5  5  5  5  7  8 10]
dist sums xy:  [ 5  6  7  8  9 10 11 12 12 12 11 11 12 13 13 14 15 15 15 15 15 14 13 12
 11 10  9  8  7  6  6  5  5  4  4  3  3  2  2  0  0  2  3  5]
dist sums xy:  [ 6  5  4  3  2  1  0  1  1  3  4  4  5  6  6  7  8  8  8  8  8  9 10 11
 12 13 14 15 16 17 17 16 16 15 15 14 14 13 13 11 11  9  8  6]
counter:  5
n_nodes:  45 (45, 2) (44, 2) corr:  0.1111111111111111


In [325]:
fig=plt.figure()

#
ax=plt.subplot(1,1,1)
img = corrs.copy()
idx = np.where(img<0.0)
img[idx] =0
# 
plt.imshow(img)
plt.colorbar()


fig = plt.figure()
selected_cell = 0
#foot1 = all_footprints[0][selected_cell]
foot1 = all_pts[0][selected_cell]
foot1 = foot1-np.median(foot1,axis=0)

idx2 = np.where(img[selected_cell]>0.3)[0]
print ("idx2: ", idx2)
#idx2 = np.arange(25)
for ctr, id_ in enumerate(idx2):
    ax=plt.subplot(5,5,ctr+1)

    # 
    foot2 = all_footprints[1][id_]
    foot2 = all_pts[1][id_]

    foot2 = foot2-np.median(foot2,axis=0)

    corr = find_neuron_correlation_footprint(foot1, foot2)
    print ("CORR: ", corr)
    
    #
    plt.scatter(foot1[:,0],
                foot1[:,1],
               alpha=.5)
    plt.scatter(foot2[:,0],
                foot2[:,1],
               alpha=.5)
    plt.title(str(selected_cell) +  '  ' + 
              str(id_)  + '    '   + 
              str(img[selected_cell,id_]))


plt.show()

idx2:  [  3   6  72 134 136 138 152]
CORR:  0.3181818181818182
CORR:  0.38636363636363635
CORR:  0.35
CORR:  0.40476190476190477
CORR:  0.4772727272727273
CORR:  0.35
CORR:  0.34


In [330]:
############################################
############################################
############################################
def get_xy_dist(cell1, cell2):
    loc1 = np.median(cell1, axis=0)
    loc2 = np.median(cell2, axis=0)
    #loc1 = np.mean(cell1, axis=0)
    #loc2 = np.mean(cell2, axis=0)
    
    dist = loc1-loc2
    return dist
    
thresh = .5
dists=[]
for c1 in trange(corrs.shape[0]):
    for c2 in range(corrs.shape[0]):
        temp = corrs[c1,c2]
        if temp>=thresh:
            dist = get_xy_dist(cells1[c1],
                               cells2[c2])
            
            dists.append(dist)

dists = np.vstack(dists)
print ('# cells matching: ', dists.shape) 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153/153 [00:00<00:00, 8355.84it/s]

# cells matching:  (69, 2)


In [331]:
###################################################
###################################################
###################################################
dists = np.vstack(dists)
print ('dists: ', dists.shape)
bin_width = 2

#
fig=plt.figure()
ax1=plt.subplot(2,2,1)
y= np.histogram(dists[:,0],bins=np.arange(-200,200,bin_width))
plt.plot(y[1][:-1],
         y[0])
argmax1 = np.argmax(y[0])
xmax1 = y[1][argmax1]
plt.scatter(y[1][argmax1],
           y[0][argmax1],
           c='red')

ax2=plt.subplot(2,2,2)
y= np.histogram(dists[:,1],bins=np.arange(-200,200,bin_width))
plt.plot(y[1][:-1],
         y[0])
argmax2 = np.argmax(y[0])
xmax2 = y[1][argmax2]
plt.scatter(y[1][argmax2],
           y[0][argmax2],
           c='red')
plt.suptitle(str(xmax1) + "   " + str(xmax2))

###############################################################
###############################################################
###############################################################

ax=plt.subplot(2,2,3)
plt.imshow(imgs[0]+imgs[1]*-1)

#
ax=plt.subplot(2,2,4)

############################################
# do initial translation
imgs1_s = imgs[0].copy()
imgs1_s = np.roll(imgs1_s, xmax1,1)
if xmax1>0:
    imgs1_s[:,:xmax1]=0
else:
    imgs1_s[:, xmax1:]=0
# 
imgs1_s = np.roll(imgs1_s, xmax2,0)
if xmax2>0:
    imgs1_s[:xmax2]=0
else:
    imgs1_s[xmax2:]=0
    
# 
plt.imshow(imgs1_s+imgs[1]*-1)


plt.show()

dists:  (69, 2)


In [ ]:
##################################################
##################################################
##################################################
data = scipy.io.loadmat('/media/cat/4TB/donato/DON-006084/'+str(sess1)+'_'+str(sess2)+'.mat')
cell_ids = data['roiMatchData']['allSessionMapping'][0][0]

refimage = data['roiMatchData']['refImage'][0][0]

# 
pts1 = data['roiMatchData']['rois'][0][0][0][1][0][0]['trans'][0][0]['fixed_out']
pts2 = data['roiMatchData']['rois'][0][0][0][1][0][0]['trans'][0][0]['moving_out']

print (pts1)
# 
matrix = cv2.findHomography(pts1, pts2)

hh = matrix[0]
print ("hh: ", hh)

# 
im = np.zeros((imgs[0].shape[0],imgs[0].shape[0],3), 
              dtype=np.uint8)
im[:,:,0] = imgs[1]
height, width, channels = im.shape
im2Reg = cv2.warpPerspective(im, hh, (width, height))

#im2Reg = np.roll(im2Reg, 100)
    

In [12]:
##################################################
##################################################
##################################################
def find_homography(points1, points2, im):
    
    # Find homography
    h, mask = cv2.findHomography(points1, points2, 
                                 cv2.RHO#cv2.RANSAC
                                )
    print ("H: ", h)
    # Use homography
    height, width, channels = im.shape
    im1Reg = cv2.warpPerspective(im, h, (width, height))

    return im1Reg, h

#
# 
im = np.zeros((imgs[0].shape[0],imgs[0].shape[0],3), dtype=np.uint8)
im[:,:,0] = imgs[1]

#
pts_1 = pts_array[0][:5000]
pts_2 = pts_array[1][:5000]
imReg, h = find_homography(pts_1,
                           pts_2, im)

fig=plt.figure()
# 
ax=plt.subplot(1,2,1)
plt.imshow(imgs[0]+imgs[1]*-1)
plt.scatter(pts_array[0][:,1],pts_array[0][:,0], c = 'white')
plt.scatter(pts_array[1][:,1],pts_array[1][:,0], c = 'red')

# 
ax=plt.subplot(1,2,2)
#plt.imshow(imgs[0]+im2Reg[:,:,0]*-1)
plt.imshow(imgs[0])


plt.scatter(pts_array[0][:,1],pts_array[0][:,0], c='blue')

from numpy.linalg import inv

# invert transform matrix, created with cv2.perspectiveTransform
M = inv(hh)  
#M = inv(h)

# 
def warp_point(xy, M):
    x= xy[0]
    y=xy[1]
    d = M[2, 0] * x + M[2, 1] * y + M[2, 2]

    return (
        int(M[0, 0] * x + M[0, 1] * y + M[0, 2] / d), # x
        int(M[1, 0] * x + M[1, 1] * y + M[1, 2] / d), # y
    )

# 
pts_trans = []
for k in range(pts_array[1].shape[0]):
    
    pts_trans.append(warp_point(pts_array[1][k], M))

pts_trans = np.vstack(pts_trans)
plt.scatter(pts_trans[:,1],
            pts_trans[:,0], c='red')


#
idx = np.where(imgs[1]>0)
idx22 = np.vstack(idx).T
print (idx22.shape)

#
img_trans = []
for k in range(idx22.shape[0]):
    img_trans.append(warp_point(idx22[k], M))

img_trans = np.vstack(img_trans)
plt.scatter(img_trans[:,1],
            img_trans[:,0],
            s=10,
            c='white')    

# 
plt.show()
#plt.imshow(imReg)


H:  [[  1.05619895   0.03023315 -46.99795151]
 [  0.05373303   1.03358448  35.25652695]
 [  0.00007159  -0.00001589   1.        ]]
(5324, 2)


In [59]:
#############################################
############ SHOW 2D FOOT PRINTS ############
#############################################

fig=plt.figure()
sparse = 3
xoffset = 0
yoffset = 0
for k in range(0, c19.stat.shape[0], sparse):
    points = np.vstack((c19.stat[k]['xpix'],
                        c19.stat[k]['ypix'])).T

    # 
    hull = ConvexHull(points)
    hull_ids = hull.vertices
    hull_points = points[hull.vertices]
    hull_points = np.vstack((hull_points, hull_points[0]))

    plt.plot(hull_points[:,0], 
             hull_points[:,1], 
             c='black',
             lw=2)

for k in range(0,c20.stat.shape[0], sparse):
    points = np.vstack((c20.stat[k]['xpix'],
                        c20.stat[k]['ypix'])).T

    # 
    hull = ConvexHull(points)
    hull_ids = hull.vertices
    hull_points = points[hull.vertices]
    hull_points = np.vstack((hull_points, hull_points[0]))

    plt.plot(hull_points[:,0]-xoffset, 
             hull_points[:,1]-yoffset, 
             c='blue',
             lw=2)
plt.show()

In [58]:
#############################################
############ SHOW 2D FOOT PRINTS ############
#############################################

fig=plt.figure()
sparse = 2
xoffset = 0
yoffset = -50
means = []
for k in range(0, c19.stat.shape[0], sparse):
    points = np.vstack((c19.stat[k]['xpix'],
                        c19.stat[k]['ypix'])).T

    means.append(np.mean(points,0))

means = np.vstack(means)
plt.scatter(means[:,0], 
           means[:,1], 
           c='black',
           s=200,
           alpha=.5)

means=[]
for k in range(0,c20.stat.shape[0], sparse):
    points = np.vstack((c20.stat[k]['xpix'],
                        c20.stat[k]['ypix'])).T

    # 
    means.append(np.mean(points,0))

means = np.vstack(means)
plt.scatter(means[:,0]+xoffset, 
           means[:,1]+yoffset, 
           c='blue',
           s=200,
           alpha=.5)
plt.show()




In [156]:
###################################################
###################################################
###################################################

plotting = True
    

dims = [512,512]
##################################################
img_19_all = np.zeros((dims[0],dims[1]))
imgs_19 = []
for k in range(len(c19.stat)):
    x = c19.stat[k]['xpix']
    y = c19.stat[k]['ypix']
    img_19_all[x,y]=c19.stat[k]['lam']
    
    # save footprint
    img_temp = np.zeros((dims[0],dims[1]))
    img_temp[x,y]=c19.stat[k]['lam']
    imgs_19.append(img_temp)
    
if plotting: 
    fig=plt.figure()
    ax=plt.subplot(121)
    plt.imshow(img_19_all)
    plt.title("P19")

###################################################
imgs_20 = []
img_20_all = np.zeros((dims[0],dims[1]))
for k in range(len(c20.stat)):
    x = c20.stat[k]['xpix']
    y = c20.stat[k]['ypix']
    img_20_all[x,y]=c20.stat[k]['lam']
    
    # save footprint
    img_temp = np.zeros((dims[0],dims[1]))
    img_temp[x,y]=c20.stat[k]['lam']
    imgs_20.append(img_temp)

if plotting:
    ax=plt.subplot(122)
    plt.imshow(img_20_all)
    plt.title("P20")
    plt.show()

    
    
imgs_19 = np.array(imgs_19)
imgs_20 = np.array(imgs_20)
print (imgs_19.shape, imgs_20.shape)

#
imgs_19_flat = imgs_19.reshape(imgs_19.shape[0],-1)
print (imgs_19_flat.shape)

imgs_20_flat = imgs_20.reshape(imgs_20.shape[0],-1)
print (imgs_20_flat.shape)

np.save('/home/cat/c19.npy',imgs_19_flat.T)
np.save('/home/cat/c20.npy',imgs_20_flat.T)
np.save('/home/cat/c19_all.npy',img_19_all)
np.save('/home/cat/c20_all.npy',img_20_all)


(531, 512, 512) (584, 512, 512)
(531, 262144)
(584, 262144)


(531, 360000)
(584, 360000)


In [155]:
##########################################################
############## LOAD DATA FROM OPS FILE ###################
##########################################################
matfile = loadmat('/media/cat/4TB/donato/DON-006084/20210519/suite2p/plane0/Fall.mat', 
                  squeeze_me=True, 
                  struct_as_record=False)

print (dir(matfile['ops']))
img = matfile['ops'].meanImg

print (img.shape)
plt.imshow(img)
plt.show()




['Lx', 'Lxc', 'Ly', 'Lyc', 'Vcorr', 'Vmap', 'Vmax', 'Vsplit', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_fieldnames', 'align_by_chan', 'allow_overlap', 'anatomical_only', 'aspect', 'baseline', 'batch_size', 'bidi_corrected', 'bidiphase', 'block_size', 'bruker', 'bruker_bidirectional', 'cellprob_threshold', 'chan2_thres', 'classifier_path', 'combined', 'connected', 'data_path', 'date_proc', 'delete_bin', 'denoise', 'diameter', 'do_bidiphase', 'do_registration', 'fast_disk', 'filelist', 'first_tiffs', 'flow_threshold', 'force_refImg', 'force_sktiff', 'frames_include', 'frames_per_file', 'frames_per_folder', 'fs', 'functional_chan', 'h5py', 'h5py_key', 'high_pass', 'ignore_f

In [ ]:
#########################################################
############ LOAD ROI MATCHED DATA VIA MATLAB ###########
#########################################################
from __future__ import print_function
import cv2
import numpy as np

               
# 
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'

clrs = ['black','red']
sessions = ['20210519', '20210520']
#sessions = ['20210520', '20210521']
#sessions = ['20210523', '20210524']

sess1 = sessions[0][-2:]
sess2 = sessions[1][-2:]

#
data = scipy.io.loadmat('/media/cat/4TB/donato/DON-006084/'+str(sess1)+'_'+str(sess2)+'.mat')
cell_ids = data['roiMatchData']['allSessionMapping'][0][0]

# convert from matlab to python data
cell_ids = cell_ids -1 
print ("# of matched cells: ", cell_ids.shape)

#fig=plt.figure()
imgs=[]
pts_array = []
all_pts = []
all_centres = []
all_footprints = []
contours_ROIMatch = []
for ctr, session in enumerate(sessions):
    
    # 
    stat = np.load(os.path.join(
                        root_dir,
                        animal_id,
                        session,
                        'suite2p/plane0/stat.npy'), allow_pickle=True)
    # 
    Fall = scipy.io.loadmat(os.path.join(
                            root_dir,
                            animal_id,
                            session,
                            'suite2p/plane0/Fall.mat'))

    # 
    iscell = np.int32(Fall['iscell'])
    print ("iscell: ", iscell.shape)

    # 
    idxs = np.where(iscell[:,0]==1)[0]
    
    print ("valid cells: ", idxs.shape, idxs[-1])
    
    img = np.zeros((512,512))
    pts3 = []
    pts4 = []
    pts5 = []
    pts6 = []
    
    # 
    
    
    # 
    all_points  = []
    for cell in cell_ids[:,ctr]:
        cell_valid = idxs[cell]
        
        # 
        points = np.vstack((stat[cell_valid]['xpix'],
                            stat[cell_valid]['ypix'])).T
        pts6.append(points)
        
        pts5.append(np.median(points,0))
        #.append(np.median(points,0))
        
        # 
        hull = ConvexHull(points)
        hull_ids = hull.vertices
        hull_points = points[hull.vertices]
        hull_points = np.vstack((hull_points, hull_points[0]))
        pts4.append(hull_points)

        

        #
        pts_local = []
        for p in range(hull_points.shape[0]-1):
            pts = np.vstack((hull_points[p],
                             hull_points[p+1]))
            pts = connect2(pts)
            img[pts[:,0],
                pts[:,1]]= 1

            pts_local.append(pts)
        pts3.append(np.vstack(pts_local))
        
        #
    
    all_footprints.append(np.array(pts6))
    pts_array.append(np.vstack(pts5))
    all_centres.append(np.vstack(pts5))
    all_pts.append(np.array(pts3))
    contours_ROIMatch.append(np.array(pts4))
    # 
    imgs.append(img)

print ("sess 1 sess 2 ", sess1, sess2)

In [ ]:
#####################################################################
########## LOAD RAW DATA AND CONTOURS FROM CALCIUM FILES ############
#####################################################################
#
import cv2
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'

clrs = ['black','red']
sessions = ['20210519', '20210520']
#sessions = ['20210520', '20210521']
#sessions = ['20210523', '20210524']

sess1 = sessions[0][-2:]
sess2 = sessions[1][-2:]

################## LOAD CONTOURS AND FOOTPRINTS ####################
centres_array = []
imgs2=[]
pts_array = []


for ctr, session in enumerate(sessions):
    root_dir = '/media/cat/4TB/donato/'
    animal_id = 'DON-006084'

    # 
    c = calcium.Calcium()
    c.root_dir = root_dir
    c.animal_id = animal_id
    c.session = session
    c.load_suite2p()
    
    print (c.F.shape)
    img = np.zeros((512,512))
    
    # 
    pts5 = []
    all_points = []
    subsample = 1
    cell_ids = np.arange(0,c.F.shape[0],subsample)
    for cell in cell_ids:
    
        # 
        points = c.get_footprint_contour(cell)
        all_points.append(points)
        
        pts5.append(np.median(points,0))
        
        #
        for p in range(points.shape[0]-1):
            pts = np.vstack((points[p],
                             points[p+1]))
            pts = connect2(pts)
            img[pts[:,0],
                pts[:,1]]= 1

    centres_array.append(np.vstack(pts5))
    imgs2.append(img)
    pts_array.append(all_points)
    
    
    

################# LOAD ROIMATCHPUB DATA #######################

data = scipy.io.loadmat('/media/cat/4TB/donato/DON-006084/'+str(sess1)+'_'+str(sess2)+'.mat')
cell_ids = data['roiMatchData']['allSessionMapping'][0][0]
# convert from matlab to python data
cell_ids = cell_ids -1 

refimage = data['roiMatchData']['refImage'][0][0]

# 
pts1 = data['roiMatchData']['rois'][0][0][0][1][0][0]['trans'][0][0]['fixed_out']
pts2 = data['roiMatchData']['rois'][0][0][0][1][0][0]['trans'][0][0]['moving_out']

print (pts1)
# 
matrix = cv2.findHomography(pts1, pts2)

hh = matrix[0]
print ("hh: ", hh)

# 
im = np.zeros((imgs[0].shape[0],imgs[0].shape[0],3), 
              dtype=np.uint8)
im[:,:,0] = imgs[1]
height, width, channels = im.shape
im2Reg = cv2.warpPerspective(im, hh, (width, height))

#im2Reg = np.roll(im2Reg, 100)
print (im2Reg.shape)

In [ ]:
#
from matplotlib.path import Path
from tqdm import tqdm, trange

#
clrs = ['black','blue','red','green']
#

#
x, y = np.meshgrid(np.arange(512), np.arange(512)) # make a canvas with coordinates
x, y = x.flatten(), y.flatten()
points_mesh = np.vstack((x,y)).T 


#
M = np.linalg.inv(hh)
ave = np.array([ 27, -55])
fig=plt.figure()
mask1 = np.zeros((512,512))
mask2 = np.zeros((512,512))
masks1 = []
masks2 = []
for ctr, session in enumerate(sessions):
    root_dir = '/media/cat/4TB/donato/'
    animal_id = 'DON-006084'

    c = calcium.Calcium()
    c.root_dir = root_dir
    c.animal_id = animal_id
    c.session = session
    c.load_suite2p()

    cell_ids = np.arange(c.F.shape[0])
    for cell in tqdm(cell_ids):
    
        # 
        points = c.get_footprint_contour(cell)
        
        if ctr==1:
            points = np.float32(points).reshape(-1,1,2)
            #print (points.shape)
            points = cv2.perspectiveTransform(points, 
                                             M).squeeze()
            #points += ave
            
        if False:
            plt.plot(points[:,1],points[:,0],
                c=clrs[ctr])
                
        p = Path(points) # make a polygon
        grid = p.contains_points(points_mesh)
        mask = grid.reshape(512,512) # now 

        if ctr==0:
            mask1+= mask
            masks1.append(mask)
        if ctr==1:
            mask2+=mask
            masks2.append(mask)
            
            
mask1 = np.clip(mask1,0,1)
mask2 = np.clip(mask2,0,1)
plt.imshow(mask1+mask2)
plt.colorbar()
#plt.xlim(0,512)
#plt.ylim(512,0)



            
plt.show()
        

In [ ]:
def find_percent_overlap(cell1, cell2):
    
    n_pix = max(np.where(cell1==1)[0].shape[0],
                np.where(cell2==1)[0].shape[0])
    
    temp = cell1+cell2
    idx = np.where(temp==2)[0]
    #print ("idx: ", idx)
    
    return idx.shape[0]/n_pix
    
masks1 = np.float32(masks1)
masks2 = np.float32(masks2)
print (masks1.shape)


#
match_array = np.zeros((masks1.shape[0], masks2.shape[0]))
for c1 in trange(masks1.shape[0]):
    for c2 in range(masks2.shape[0]):
        cell1 = masks1[c1]
        cell2 = masks2[c2]
        temp = np.max(cell1+cell2)
        #print ("temp: ", temp)
        if temp<2:
            continue
        
        idx = np.where(cell1>0)
        res = find_percent_overlap(cell1, cell2)
        match_array[c1,c2]=res
        #print ("percent overalp: ", res)
        #break
    #break

In [ ]:
############################################
thresh = 0.6

img = np.zeros((512,512))
sess1_ctr = np.zeros((match_array.shape[0]))
sess2_ctr = np.zeros((match_array.shape[1]))
cell1_ids = []
cell2_ids = []
for c1 in range(match_array.shape[0]):
    for c2 in range(match_array.shape[1]):
        temp = match_array[c1,c2]
        if temp>=thresh:
            temp1 = masks1[c1]
            temp2 = masks2[c2]
            cell1_ids.append(c1)
            cell2_ids.append(c2)
            img+= temp1
            img+= temp2*2
            sess1_ctr[c1]=1
            sess2_ctr[c2]=1

img = np.clip(img, 0, 4)
      
########################################
fig=plt.figure()
plt.style.use('default')

ax=plt.subplot(1,2,1)
idx = np.where(img==0)
img[idx]=np.nan
img = plt.imshow(img,
          cmap='Set1')

cbar = plt.colorbar(img)

labels = ['cell #1', 'cell #2', '2cell overlap', '3+ cell overlap']
cbar.set_ticks([1,2,3,4])
cbar.set_ticklabels(labels)

plt.xlim(0,512)
plt.ylim(512,0)


############################################################
ax=plt.subplot(1,2,2)

def plot_contours_transformed_session_cell_ids(M,
                                               transform_flag,
                                               session,
                                              cell_ids,
                                              clr):
    root_dir = '/media/cat/4TB/donato/'
    animal_id = 'DON-006084'

    c = calcium.Calcium()
    c.root_dir = root_dir
    c.animal_id = animal_id
    c.session = session
    c.load_suite2p()

    for k in cell_ids:
        points = c.get_footprint_contour(k)
        if transform_flag:
            points = points.astype('float32').reshape(-1,1,2)
            points = cv2.perspectiveTransform(points, 
                                             M).squeeze()
        plt.plot(points[:,0],
                points[:,1],
                c=clr,
                linewidth=2,
                alpha=.7)
    
plot_contours_transformed_session_cell_ids(M,False,
                                       sessions[0],
                                      cell1_ids,
                                      'white')
plot_contours_transformed_session_cell_ids(M,True,
                                           sessions[1],
                                          cell2_ids,
                                          'red')
ax.set_facecolor("black")
plt.xlim(0,512)
plt.ylim(512,0)

plt.suptitle("Min overlap between 2 cells (threshold): "+ str(thresh*100) +   "%\n # cells sess1: "+str(np.sum(sess1_ctr))+" , # cells sess 2 "+str(np.sum(sess2_ctr)))
plt.show()